In [1]:
just_checking_integrity=True

In [2]:
import gc
import pandas as pd
import scipy.sparse as sps

In [3]:
valid=sps.load_npz("../dataset/processed_data/URM_valid_bought.npz")
bought=valid.indices

In [4]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"]
for file in ["UCF","ICBF","Graph","ICF"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"../dataset/candidates/traditional_recs/train/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df
        

#del dataset_max
del df_feats
del df_max

63
loading UCF
1399885
loaded UCF
0
loading ICBF
1268640
loaded ICBF
0
loading Graph
818193
loaded Graph
0
loading ICF
1617531
loaded ICF


In [5]:
import ast
submission_df=pd.read_csv("../dataset/candidates/NN/GRU/train/candidates.csv")
if just_checking_integrity:
    submission_df["score"]=submission_df["score"].apply(lambda x:x.replace("-inf","0"))
    submission_df["score"]=submission_df["score"].map(ast.literal_eval)
else:
    submission_df["score"]=submission_df["score"].map(ast.literal_eval)
submission_df["code"]=submission_df["code"].map(ast.literal_eval)
submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
submission_df = submission_df.explode(['code','score', 'rank'])
submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [6]:
submission_df.head()

,Session_Id,Item_ID,score_gru,rank_gru
0,918743,2276,0.000145,1
0,918743,1609,0.000123,2
0,918743,4167,0.000116,3
0,918743,2694,0.000114,4
0,918743,554,0.000101,5


In [7]:
import ast
submission_df_tr=pd.read_csv("../dataset/candidates/NN/Transformer/train/candidates.csv")
if just_checking_integrity:
    submission_df_tr["score"]=submission_df_tr["score"].apply(lambda x:x.replace("-inf","0"))
    submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
else:
    submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [8]:
import ast
submission_df_LSTM=pd.read_csv("../dataset/candidates/NN/LSTM/train/candidates.csv")
if just_checking_integrity:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].apply(lambda x:x.replace("-inf","0"))
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
else:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [9]:
dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")

dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")


In [10]:
if just_checking_integrity:
    dataset=dataset[:int(0.5*len(dataset))]

In [11]:
del dataset_max
del submission_df
del submission_df_tr

In [12]:
import numpy as np
sessions=np.unique(dataset["Session_Id"])
train=pd.DataFrame()
train["Session_Id"]=sessions
if just_checking_integrity:
    train["Item_ID"]=bought[:len(sessions)]
else:
    train["Item_ID"]=bought
train["target"]=1

In [13]:
gc.collect()

240

In [14]:
dataset=dataset.merge(train,on=["Session_Id","Item_ID"],how="left")
dataset.fillna(0,inplace=True)
gc.collect()

23

In [15]:
dataset["target"]=dataset["target"].astype("int8")
gc.collect()

23

In [16]:
sessions_containing_bought_among_candidates=dataset[["Session_Id","target"]].groupby("Session_Id").agg(sum).reset_index()
sessions_containing_bought_among_candidates=sessions_containing_bought_among_candidates.loc[sessions_containing_bought_among_candidates["target"]>0]

dataset=dataset[dataset['Session_Id'].isin(sessions_containing_bought_among_candidates["Session_Id"].tolist())]

In [17]:
dataset.head(20)

,Session_Id,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,score_gru,rank_gru,score_transformer,rank_transformer,score_LSTM,rank_LSTM,Max_Score_UCF,Max_Score_ICBF,Max_Score_Graph,Max_Score_ICF,target
215,18383,1,0.031490,0.000000,0.007877,5.231782e-07,0,0.0,0,0.0,0,0.0,178.3819,7.204127,8.918959,0.744729,0
216,18383,55,0.086116,0.632362,0.032122,2.700396e-02,0,0.0,0,0.0,0,0.0,178.3819,7.204127,8.918959,0.744729,0
217,18383,215,0.062864,0.000000,0.006708,0.000000e+00,0,0.0,0,0.0,0,0.0,178.3819,7.204127,8.918959,0.744729,0
218,18383,227,0.026399,0.000000,0.000000,0.000000e+00,0,0.0,0,0.0,0,0.0,178.3819,7.204127,8.918959,0.744729,0
219,18383,242,0.064825,0.000000,0.000003,1.907103e-06,0,0.0,0,0.0,0,0.0,178.3819,7.204127,8.918959,0.744729,0
220,18383,292,0.033530,0.000000,0.000000,0.000000e+00,0,0.0,0,0.0,0,0.0,178.3819,7.204127,8.918959,0.744729,0
221,18383,320,0.053880,0.000000,0.013578,1.130169e-02,0,0.0,0,0.0,0,0.0,178.3819,7.204127,8.918959,0.744729,0
222,18383,342,0.030933,0.000000,0.007783,0.000000e+00,0,0.0,0,0.0,0,0.0,178.3819,7.204127,8.918959,0.744729,0
223,18383,357,0.043342,0.000000,0.000000,0.000000e+00,0,0.0,0,0.0,0,0.0,178.3819,7.204127,8.918959,0.744729,0
224,18383,379,0.087276,0.637329,0.033344,2.840736e-02,0,0.0,0,0.0,0,0.0,178.3819,7.204127,8.918959,0.744729,0


In [18]:
dataset

,Session_Id,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,score_gru,rank_gru,score_transformer,rank_transformer,score_LSTM,rank_LSTM,Max_Score_UCF,Max_Score_ICBF,Max_Score_Graph,Max_Score_ICF,target
215,18383,1,0.031490,0.000000,0.007877,5.231782e-07,0,0.0,0,0.0,0,0.0,178.38190,7.204127,8.918959,0.744729,0
216,18383,55,0.086116,0.632362,0.032122,2.700396e-02,0,0.0,0,0.0,0,0.0,178.38190,7.204127,8.918959,0.744729,0
217,18383,215,0.062864,0.000000,0.006708,0.000000e+00,0,0.0,0,0.0,0,0.0,178.38190,7.204127,8.918959,0.744729,0
218,18383,227,0.026399,0.000000,0.000000,0.000000e+00,0,0.0,0,0.0,0,0.0,178.38190,7.204127,8.918959,0.744729,0
219,18383,242,0.064825,0.000000,0.000003,1.907103e-06,0,0.0,0,0.0,0,0.0,178.38190,7.204127,8.918959,0.744729,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276252,24409,3458,0.145934,0.146330,0.082520,2.148758e-01,0,0.0,0,0.0,0,0.0,75.80949,11.361944,4.000053,1.008071,0
1276253,24409,3461,0.160343,0.635567,0.106817,5.759829e-01,0,0.0,0,0.0,0,0.0,75.80949,11.361944,4.000053,1.008071,0
1276254,24409,3497,0.107480,0.000000,0.018200,0.000000e+00,0,0.0,0,0.0,0,0.0,75.80949,11.361944,4.000053,1.008071,0
1276255,24409,3550,0.106927,0.000000,0.000000,0.000000e+00,0,0.0,0,0.0,0,0.0,75.80949,11.361944,4.000053,1.008071,0


In [19]:
dataset_macro_features = pd.read_csv('../dataset/processed_data/macro_feats_NN.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')
del dataset_macro_features

In [20]:
#dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [21]:
from sklearn.model_selection import KFold
import lightgbm
import os
k=10


score=0
kf = KFold(n_splits=k)
index=0
for train_index, valid_index in kf.split(train):

    train_sessions=train["Session_Id"][train_index].tolist()
    valid_sessions=train["Session_Id"][valid_index].tolist()

    train_set=dataset[dataset["Session_Id"].isin(train_sessions)]
    valid_set=dataset[dataset["Session_Id"].isin(valid_sessions)]

    train_set=train_set.sort_values("Session_Id")
    valid_set=valid_set.sort_values("Session_Id")
    
    qids_train = train_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()
    qids_valid = valid_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()
    
    X_train=train_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_train=train_set["target"]
    
    X_valid=valid_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_valid=valid_set["target"]

    model = lightgbm.LGBMRanker(
    objective="lambdarank",
    #objective="rank_xendcg",
    #metric="mrr",
    n_estimators=2000,
    lambdarank_truncation_level=103,
    reg_alpha=1e-4,
    learning_rate=0.05,
    min_child_samples=400,
    subsample=0.8,
    subsample_freq=20
    )
    ranker=model.fit(
        X=X_train,
        y=y_train,
        group=qids_train,
        eval_set=[(X_valid, y_valid)],
        eval_group=[qids_valid],
        eval_at=(100),
        callbacks=[lightgbm.log_evaluation(),lightgbm.early_stopping(20,first_metric_only=True)],
        #verbose=10,
        eval_metric=["map","ndcg"]
    )
    ranker.booster_.save_model(f'../rankers/model_{index}.txt')
    #Get maximum val score
    score+=np.max(ranker._evals_result["valid_0"]["map@100"])
    index+=1

[1]	valid_0's map@100: 0.233222	valid_0's ndcg@100: 0.37276
Training until validation scores don't improve for 20 rounds
[2]	valid_0's map@100: 0.252296	valid_0's ndcg@100: 0.389726
[3]	valid_0's map@100: 0.261492	valid_0's ndcg@100: 0.397567
[4]	valid_0's map@100: 0.268346	valid_0's ndcg@100: 0.402891
[5]	valid_0's map@100: 0.264352	valid_0's ndcg@100: 0.399745
[6]	valid_0's map@100: 0.262914	valid_0's ndcg@100: 0.398849
[7]	valid_0's map@100: 0.262694	valid_0's ndcg@100: 0.399247
[8]	valid_0's map@100: 0.264268	valid_0's ndcg@100: 0.400134
[9]	valid_0's map@100: 0.262757	valid_0's ndcg@100: 0.399127
[10]	valid_0's map@100: 0.262396	valid_0's ndcg@100: 0.398938
[11]	valid_0's map@100: 0.263992	valid_0's ndcg@100: 0.400061
[12]	valid_0's map@100: 0.266864	valid_0's ndcg@100: 0.402705
[13]	valid_0's map@100: 0.266534	valid_0's ndcg@100: 0.402494
[14]	valid_0's map@100: 0.265512	valid_0's ndcg@100: 0.401842
[15]	valid_0's map@100: 0.264713	valid_0's ndcg@100: 0.400799
[16]	valid_0's map@

In [22]:
final_performcance=score/k
print("overall MAP@100 = ", final_performcance)

overall MAP@100 =  0.27496084158275147
